In [ ]:
# 🚀 AI Text Checker – 💥 Valentin Djillas - https://www.linkedin.com/in/valentin-djillas-575406112/

!pip install --upgrade openai google-api-python-client google-auth-httplib2 google-auth-oauthlib --quiet

import openai
from google.colab import auth
from googleapiclient.discovery import build
import re
import matplotlib.pyplot as plt
import random


OPENAI_API_KEY = "[добавете вашия API KEY]"

openai.api_key = OPENAI_API_KEY
auth.authenticate_user()


def extract_doc_id(url):
    match = re.search(r'/d/([a-zA-Z0-9-_]+)', url)
    return match.group(1) if match else None

def get_text_from_google_doc(doc_id):
    docs_service = build('docs', 'v1')
    doc = docs_service.documents().get(documentId=doc_id).execute()
    content = doc.get('body').get('content')
    text = ""
    for item in content:
        paragraph = item.get("paragraph")
        if paragraph:
            for elem in paragraph.get("elements", []):
                text += elem.get("textRun", {}).get("content", "")
    return text.strip()

def analyze_with_gpt4(prompt_text):
    try:
        client = openai.OpenAI(api_key=OPENAI_API_KEY)
        response = client.chat.completions.create(
            model="gpt-4o",
            messages=[{"role": "user", "content": prompt_text}],
            temperature=0.5,
            max_tokens=2000
        )
        return response.choices[0].message.content.strip()
    except Exception as e:
        return f"GPT анализ неуспешен: {e}"

def build_analysis_prompt(text):
    return f'''
🔍 Анализирай текста според най-добрите SEO и E-E-A-T практики:

1️⃣ AI вероятност (%):
(дай точен процент между 0% и 100% с ясна обосновка)

2️⃣ Признаци за AI-генериране:
(посочи конкретни проблемни фрази и повтарящи се изрази)

3️⃣ E-E-A-T Анализ (по проценти и детайлна обосновка за всяка категория):
- Experience:
- Expertise:
- Authoritativeness:
- Trustworthiness:

4️⃣ Конкретни препоръки за значително подобрение на E-E-A-T:
- Посочи конкретни примери, лични истории или реален опит, които могат да бъдат добавени към текста.
- Предложи поне 2 специфични идеи за експертни цитати или технически детайли, които да направят текста авторитетен.
- Посочи поне 1 външен източник или тип клиентски отзив, който би бил подходящ за повишаване на доверието.

5️⃣ Емоционална ангажираност (% и препоръки):
- Дай конкретна идея за кратка лична или емоционална история, която би повишила ангажираността.

6️⃣ Search Intent:
(Informational, Commercial, Transactional, Navigational)

7️⃣ Основни SEO ключови думи (изброй до 5 важни ключови думи):

✍️ Rewrite на текста с всички предложени подобрения:
- Добави предложените лични примери, експертни цитати и източници.
- Включи емоционалната история или пример, който предлагаш.
- Допусни максимум 2-3 минимални и реалистични печатни или пунктуационни грешки за хуманизация.
- Подчертай новите неща, които предлагаш за добавяне в текста, за да се знае кое е ново.

Оригинален текст:
"""{text}"""
'''

def extract_percentage(text, keyword):
    pattern = rf"{keyword}.*?(\d{{1,3}})%"
    match = re.search(pattern, text, re.IGNORECASE | re.DOTALL)
    return int(match.group(1)) if match else 0

def show_single_bar(value, label, title, color):
    fig, ax = plt.subplots(figsize=(7, 1.5))
    ax.barh([0], [value], color=color)
    ax.set_xlim(0, 100)
    ax.set_yticks([])
    ax.set_xticks([0, 25, 50, 75, 100])
    ax.set_title(f"{title}: {value}%", fontsize=12, weight='bold')
    ax.axvline(50, color='gray', linestyle='--', linewidth=1)
    ax.axvline(75, color='gray', linestyle='--', linewidth=1)
    for spine in ['top', 'right', 'left']:
        ax.spines[spine].set_visible(False)
    plt.show()

def show_eeat_chart(values):
    labels = ["Experience", "Expertise", "Authority", "Trust"]
    colors = ["skyblue", "dodgerblue", "steelblue", "navy"]
    fig, ax = plt.subplots(figsize=(8, 2))
    bars = ax.bar(labels, values, color=colors)
    ax.set_ylim(0, 100)
    ax.set_title("E-E-A-T Анализ", fontsize=12, fontweight='bold')
    for bar, val in zip(bars, values):
        ax.text(bar.get_x() + bar.get_width()/2, val + 2, f"{val}%", ha='center', va='bottom')
    plt.show()

def humanize_text(text):
    words = text.split()
    for i in range(len(words)):
        if random.random() < 0.005:
            words[i] += random.choice(['.', ','])
        elif random.random() < 0.003:
            words[i] = words[i].capitalize()
    return " ".join(words)

links_input = input("Въведи Google Docs линк(ове):\n")
links = [l.strip() for l in re.split(r'[\n,]+', links_input) if l.strip()]

if len(links) == 0:
    print("❌ Няма въведени валидни линкове.")
else:
    for idx, url in enumerate(links):
        doc_id = extract_doc_id(url)
        if not doc_id:
            print(f"❌ Невалиден линк: {url}")
            continue
        text = get_text_from_google_doc(doc_id)
        print(f"\n📄 Документ {idx+1}: {url}")
        print("Извлечен текст (начало):\n", text[:1000])

        prompt = build_analysis_prompt(text)
        analysis_result = analyze_with_gpt4(prompt)

        print("\n🔍 Анализ:\n", analysis_result)

        ai_prob = extract_percentage(analysis_result, "AI вероятност")
        emotion_prob = extract_percentage(analysis_result, "Емоционална ангажираност")

        eeat_values = [
            extract_percentage(analysis_result, "Experience"),
            extract_percentage(analysis_result, "Expertise"),
            extract_percentage(analysis_result, "Authoritativeness"),
            extract_percentage(analysis_result, "Trustworthiness")
        ]

        show_single_bar(ai_prob, "AI вероятност", "🤖 AI вероятност", "crimson")
        show_single_bar(emotion_prob, "Емоции", "🎭 Емоционална ангажираност", "orange")
        show_eeat_chart(eeat_values)

        if "Rewrite с подобрения:" in analysis_result:
            rewrite = analysis_result.split("Rewrite с подобрения:")[-1].strip()
            print("\n✍️ Пренаписан текст с подобрения:\n", humanize_text(rewrite))